In [1]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from datetime import timedelta
from typing import Optional

In [2]:
def get_historical_data(symbol: str, last_n: int):
    try:
        # Connect to MongoDB
        client = MongoClient("mongodb://127.0.0.1:27017/")
        db = client["stockdata"]
        collection = db[symbol]

        # Retrieve the last N documents for the given symbol
        cursor = collection.find().sort("_id", -1).limit(last_n)
        data_list = list(cursor)

        # Convert the selected documents into a DataFrame
        df = pd.DataFrame(data_list)

        return df

    finally:
        # Close the MongoDB connection
        client.close()

In [3]:
df = get_historical_data('TURSG.IS',50)

In [4]:
df.head()

,_id,Open,High,Low,Close,Adj Close,Volume
0,2024-03-15,54.849998,55.599998,53.900002,54.200001,54.200001,10664429
1,2024-03-14,53.099998,56.150002,52.849998,54.849998,54.849998,4506471
2,2024-03-13,52.250000,54.349998,51.049999,53.250000,53.250000,3520302
3,2024-03-12,52.650002,53.099998,51.500000,52.250000,52.250000,2099522
4,2024-03-11,54.500000,55.000000,52.200001,52.650002,52.650002,4700633


In [5]:
import plotly.express as px
from pymongo import MongoClient
import pandas as pd

def plot_stock(symbol, last_n):
    try:
        client = MongoClient("mongodb://127.0.0.1:27017/")
        db = client["stockdata"]
        collection = db[symbol]

        cursor = collection.find().sort("_id", -1).limit(last_n)
        data_list = list(cursor)

        df = pd.DataFrame(data_list)

        df["_id"] = pd.to_datetime(df["_id"])

        fig = px.line(df, x="_id", y=["Open", "High", "Low", "Close"], 
                      labels={"_id": "Date", "value": "Price"},
                      title=f"Stock Price Over Time {symbol}")

        fig.update_layout(
            xaxis_title="Date",
            yaxis_title="Price",
            legend_title="Type",
            hovermode="x",
            template="plotly_dark",
            width = 1500,
            height = 750
        )

        fig.show()

    finally:
        client.close()



In [10]:
plot_stock('ASELS.IS',365)